# Making a dataset

In [2]:
from pathlib import Path
import requests
import unicodedata

In [3]:
def get_gutenberg_book(
	id: int|None = 84,
	data_temp: Path|str = "../data/gutenberg_data",
	remove_gutenberg_meta: bool = True,
) -> str:
	
	data_temp = Path(data_temp)
	data_temp.mkdir(parents=True, exist_ok=True)
	
	url: str = f"https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt"
	data_path: Path = Path(data_temp) / f"{id}.txt"
	data: str
	# read from cache if it exists
	if data_path.exists():
		with open(data_path, 'r', encoding='utf-8') as file:
			data = file.read()
	else:
		# download if it doesn't exist
		response = requests.get(url)
		response.raise_for_status()  # Ensure that the download was successful
		data = response.text

		# save to cache
		with open(data_path, 'w', encoding='utf-8') as file:
			file.write(data)

	# remove header/footer
	if remove_gutenberg_meta:
		data = '***'.join(data.split('***')[2:])
		data = '***'.join(data.split('***')[:-1])
	
	return data

def get_many_books(
		ids: list[int],
		data_temp: Path|str = "../data/gutenberg_data",
	) -> list[str]:
	
	data: list[str] = []
	for id in ids:
		print(f"Getting book {id}...")
		item: str = get_gutenberg_book(id, data_temp)
		print(f"\t{len(item)} characters read")
		data.append(item)
	
	return data

In [16]:
def process_text(
	text: str,
	allowed_punctuation: str = "-.,;:!?()\"" + "".join(str(x) for x in range(10)),
	punctuation_convert: dict[str,str] = {'—': '-'},
	numbers_allowed: bool = True,
) -> str:
	
	# replace some special characters which unicode won't normalize properly
	for char, replacement in punctuation_convert.items():
		text = text.replace(char, replacement)

	# if a line has ".jpg" in it, remove that line (this is specific to Don Quixote)
	text = '\n'.join(
		line 
		for line in text.split('\n')
		if '.jpg' not in line
	)

	# Normalize the string to decompose Unicode characters
	text = unicodedata.normalize('NFKD', text)

	# Encode to ASCII bytes, then decode back to string, ignoring errors
	text = text.encode('ascii', 'ignore').decode('ascii')

	# remove newlines and tabs
	text = text.replace('\n', ' ').replace('\t', ' ')


	# put spaces around allowed punctuation
	for char in allowed_punctuation:
		text = text.replace(char, f' {char} ')


	# remove leading and trailing spaces
	text = text.strip()

	# remove multiple spaces
	while '  ' in text:
		text = text.replace('  ', ' ')


	# remove all characters except (alphanumeric, allowed_punctuation, ' ')
	text = ''.join(
		(
			char 
			if (
				(char.isalnum() and numbers_allowed) or (char.isalpha)
				or char in allowed_punctuation 
				or char == ' '
			)
			else ' '
		)
		for char in text 
	)

	# convert to lowercase
	text = text.lower()

	text = text.strip()

	return text


In [17]:
def tokenize(
	text: str,
	process: bool = False,
) -> list[str]:
	if process:
		text = process_text(text)
	return text.split(' ')

In [18]:

DATA_RAW: list[str] = get_many_books([84, 15, 18, 82, 996, 2600])
DATA: str = " ".join(process_text(x, allowed_punctuation="", numbers_allowed=False) for x in DATA_RAW)

#print(DATA[:1000])

DATA_TOKENIZED: list[str] = tokenize(DATA)
TOKEN_SET: set[str] = set(DATA_TOKENIZED)
TOKEN_ALPHABET: list[str] = sorted(list(TOKEN_SET))
TOKEN_TO_INDEX: dict[str, int] = {token: i for i, token in enumerate(TOKEN_ALPHABET)}
#INDEX_TO_TOKEN: dict[int, str] = {i: token for i, token in enumerate(TOKEN_ALPHABET)}
MODEL_DATA: list[int] = [TOKEN_TO_INDEX[token] for token in DATA_TOKENIZED]


Getting book 84...
	426785 characters read
Getting book 15...
	1241025 characters read
Getting book 18...
	1192776 characters read
Getting book 82...
	1124986 characters read
Getting book 996...
	2342262 characters read
Getting book 2600...
	3273998 characters read


In [19]:
print(DATA_TOKENIZED[:1000])

['frankenstein;', 'or,', 'the', 'modern', 'prometheus', 'by', 'mary', 'wollstonecraft', '(godwin)', 'shelley', 'contents', 'letter', '1', 'letter', '2', 'letter', '3', 'letter', '4', 'chapter', '1', 'chapter', '2', 'chapter', '3', 'chapter', '4', 'chapter', '5', 'chapter', '6', 'chapter', '7', 'chapter', '8', 'chapter', '9', 'chapter', '10', 'chapter', '11', 'chapter', '12', 'chapter', '13', 'chapter', '14', 'chapter', '15', 'chapter', '16', 'chapter', '17', 'chapter', '18', 'chapter', '19', 'chapter', '20', 'chapter', '21', 'chapter', '22', 'chapter', '23', 'chapter', '24', 'letter', '1', '_to', 'mrs.', 'saville,', 'england._', 'st.', 'petersburgh,', 'dec.', '11th,', '17-.', 'you', 'will', 'rejoice', 'to', 'hear', 'that', 'no', 'disaster', 'has', 'accompanied', 'the', 'commencement', 'of', 'an', 'enterprise', 'which', 'you', 'have', 'regarded', 'with', 'such', 'evil', 'forebodings.', 'i', 'arrived', 'here', 'yesterday,', 'and', 'my', 'first', 'task', 'is', 'to', 'assure', 'my', 'dear'